In [2]:
 !pip3 install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 46.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=1ef0302821581ee18eae37624e0aea8a4dd67a4a1366fa95bb3694c6235d8100
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc1


In [0]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [4]:
tf.__version__

'2.1.0'

In [5]:
os.environ['KAGGLE_USERNAME'] = "arghyabiswas0" # username from the json file
os.environ['KAGGLE_KEY'] = "b34716e2e8673d257783d166884a382d" # key from the json file
!kaggle datasets download -d grassknoted/asl-alphabet # api copied from kaggle
from zipfile import ZipFile
file_name = "asl-alphabet.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

 98% 1.01G/1.03G [00:28<00:00, 44.1MB/s]
100% 1.03G/1.03G [00:28<00:00, 38.2MB/s]
Done


In [0]:
train_dir = 'asl_alphabet_train/asl_alphabet_train'
test_dir = 'asl_alphabet_test/asl_alphabet_test'

In [7]:
def load_unique():
    size_img = 64,64 
    labels_for_plot = []
    for folder in os.listdir(train_dir):
        for file in os.listdir(train_dir + '/' + folder):
            filepath = train_dir + '/' + folder + '/' + file
            break
    return labels_for_plot

labels_for_plot = load_unique()
print("unique_labels = ", labels_for_plot)

unique_labels =  ['O', 'B', 'space', 'J', 'nothing', 'W', 'Z', 'T', 'del', 'R', 'I', 'X', 'F', 'Q', 'M', 'C', 'G', 'D', 'P', 'E', 'Y', 'N', 'U', 'V', 'K', 'L', 'S', 'A', 'H']


In [0]:
labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26,'del':27,'nothing':28}

def load_data():
    images = []
    labels = []
    size = 64,64
    print("LOADING DATA FROM : ",end = "")
    for folder in os.listdir(train_dir):
        print(folder, end = ' | ')
        for image in os.listdir(train_dir + "/" + folder):
            temp_img = cv2.imread(train_dir + '/' + folder + '/' + image)
            temp_img = cv2.resize(temp_img, size)
            images.append(temp_img)
            labels.append(labels_dict[folder])
    images = np.array(images)
    images = images.astype('float32')/255.0
    labels = tf.keras.utils.to_categorical(labels)
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.05)
    print()
    print('Loaded', len(X_train),'images for training,','Train data shape =',X_train.shape)
    print('Loaded', len(X_test),'images for testing','Test data shape =',X_test.shape)
    return X_train, X_test, Y_train, Y_test

In [9]:
X_train, X_test, Y_train, Y_test = load_data()

LOADING DATA FROM : O | B | space | J | nothing | W | Z | T | del | R | I | X | F | Q | M | C | G | D | P | E | Y | N | U | V | K | L | S | A | H | 
Loaded 82650 images for training, Train data shape = (82650, 64, 64, 3)
Loaded 4350 images for testing Test data shape = (4350, 64, 64, 3)


In [15]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(16, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (64,64,3)))
model.add(tf.keras.layers.Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = [3,3]))

model.add(tf.keras.layers.Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = [3,3]))

model.add(tf.keras.layers.Conv2D(128, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(tf.keras.layers.Conv2D(256, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = [3,3]))

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(29, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ["accuracy"])

model.fit(X_train, Y_train, batch_size = 64, epochs = 10, validation_split = 0.2)

Train on 66120 samples, validate on 16530 samples
Epoch 1/10
66120/66120 [==============================] - 12s 182us/sample - loss: 0.9215 - accuracy: 0.7951 - val_loss: 0.2454 - val_accuracy: 0.9805
Epoch 2/10
66120/66120 [==============================] - 11s 173us/sample - loss: 0.1970 - accuracy: 0.9781 - val_loss: 0.2470 - val_accuracy: 0.9541
Epoch 3/10
66120/66120 [==============================] - 11s 173us/sample - loss: 0.1723 - accuracy: 0.9801 - val_loss: 0.1645 - val_accuracy: 0.9786
Epoch 4/10
66120/66120 [==============================] - 12s 174us/sample - loss: 0.1534 - accuracy: 0.9824 - val_loss: 0.3981 - val_accuracy: 0.9290
Epoch 5/10
66120/66120 [==============================] - 12s 174us/sample - loss: 0.1350 - accuracy: 0.9898 - val_loss: 0.0625 - val_accuracy: 0.9991
Epoch 6/10
66120/66120 [==============================] - 12s 178us/sample - loss: 0.1474 - accuracy: 0.9860 - val_loss: 0.1437 - val_accuracy: 0.9893
Epoch 7/10
66120/66120 [====================

### The following code segment is to convert the Tensorflow model that we trained into a TFLite model so that it can be implemented in an Android app.

In [0]:
import tensorflow_hub as hub

In [12]:
ASL_SAVED_MODEL = "saved_models/asl_aplhabet"
tf.keras.models.save_model(model, ASL_SAVED_MODEL,overwrite=True,
    include_optimizer=True,
    save_format='tf',
    signatures=None,
    options=None)
asl_model = tf.keras.models.load_model(ASL_SAVED_MODEL,custom_objects={'KerasLayer':hub.KerasLayer},
    compile=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_models/asl_aplhabet/assets


In [0]:
!mkdir "tflite_models"
TFLITE_MODEL = "tflite_models/asl.tflite"
TFLITE_QUANT_MODEL = "tflite_models/asl_quant.tflite"

In [14]:
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : asl_model(x))
# Save the concrete function.
concrete_func = run_model.get_concrete_function(
tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)
# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)
# Convert the model to quantized version with post-training quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
open(TFLITE_QUANT_MODEL, "wb").write(tflite_quant_model)
print("TFLite models and their sizes:")
!ls "tflite_models" -lh

TFLite models and their sizes:
total 4.6M
-rw-r--r-- 1 root root 932K Mar 30 07:32 asl_quant.tflite
-rw-r--r-- 1 root root 3.6M Mar 30 07:32 asl.tflite
